# Supervised Models

This notebook tests a variety of classification algorithms on our finalized dataset of reviews.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from collections import Counter
import itertools
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from nltk.stem import WordNetLemmatizer 
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD

In [2]:
# import csv files as dataframes
df_engineered = pd.read_csv('df_engineered.csv') # these are our engineered features
df_reviews = pd.read_csv('final_df.csv') # this contains our corpus 

In [3]:
# concatenate the dataframes into one final dataframe
df_final = pd.concat([df_engineered[['review_length','word_count','exclamation_count','question_count']],
                      df_reviews],axis=1)

In [4]:
df_final.head()

,review_length,word_count,exclamation_count,question_count,cleaned,overall
0,2175,446,0,2,professional otr truck driver buy tnd 700 truc...,1.0
1,254,49,0,0,mount work really well get go problem arm litt...,4.0
2,975,168,0,1,process turn master bedroom fortress solitude ...,5.0
3,1048,209,0,0,give mount 3 star 5 start box mount wall reall...,3.0
4,435,80,0,0,buy mount 30 34 dell u3011 monitor fit dell qu...,4.0


In [5]:
# split data into train and test set
X = df_final[['cleaned','review_length','word_count',
              'exclamation_count','question_count']]
y = df_final['overall']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [ ]:
X_engineered = X_train[['review_length','word_count',
              'exclamation_count','question_count',
                   'review_length']]

In [ ]:
X_test_engineered = X_test[['review_length','word_count',
              'exclamation_count','question_count',
                   'review_length']]

## Vectorize

#### TF-IDF

In [6]:
# vectorize the train set
tf_idf = TfidfVectorizer(max_df=0.5, max_features=10000,
                                  min_df=2, token_pattern = '(?ui)\\b\\w*[a-z]+\\w*\\b', analyzer='word')
tf_idf_vectors = tf_idf.fit_transform(X_train['cleaned'].values.astype('U'))

In [7]:
# put into dataframe so we can run it through a classification algorithm
vector_df = pd.DataFrame(tf_idf_vectors.toarray(), columns=tf_idf.get_feature_names())

In [8]:
# vectorize the test set
tfidf_test = tf_idf.transform(X_test['cleaned'].values.astype('U'))

In [9]:
# put into dataframe
vector_df_test = pd.DataFrame(tfidf_test.toarray(), columns=tf_idf.get_feature_names())

#### Bag of words

In [ ]:
# vectorize the train set, this time with the bag of words method
bow = CountVectorizer(max_df=0.5, max_features=10000,
                                  min_df=2, token_pattern = '(?ui)\\b\\w*[a-z]+\\w*\\b', analyzer='word')

In [ ]:
bow_vectors = bow.fit_transform(X_train['reviews'])

In [ ]:
# put into dataframe
bow_df = pd.DataFrame(bow_vectors.toarray(), columns=bow.get_feature_names())

In [ ]:
# vectorize test set too
bow_vectors_test = bow.transform(X_test['reviews'])

In [ ]:
bow_test_df = pd.DataFrame(bow_vectors_test.toarray(), columns=bow.get_feature_names())

### SVD

We tried using SVD to reduce our feature space so we could use it in conjuction with our engineered features. We did not have the computing power to merge the engineered features with the TF-IDF dataframe.

In [ ]:
# use SVD to reduce dimensionality of feature space
svd = TruncatedSVD(n_components=100)
LSA = svd.fit_transform(tf_idf_vectors)

In [ ]:
df_LSA = pd.DataFrame(LSA)

In [ ]:
# check out much of the variance is explained by the principal components
svd.explained_variance_ratio_.sum()

In [ ]:
# transform test set too
LSA_test = svd.transform(tfidf_test)

In [ ]:
df_LSA_test = pd.DataFrame(LSA_test)

In [ ]:
## concatenate vectorized dataframe to dataframe of engineered features

In [ ]:
X_engineered.reset_index(inplace=True)

In [ ]:
df_SVD_train = pd.concat([df_LSA,X_engineered], axis=1, ignore_index=True)

In [ ]:
## concatenate vectorized TEST dataframe to OG TEST dataframe with engineered feats.

In [ ]:
X_test_engineered.reset_index(inplace=True)

In [ ]:
df_SVD_test = pd.concat([df_LSA_test,X_test_engineered], axis=1)

## Modeling

In [10]:
def print_metrics(labels, preds):
    print("Precision Score: {}".format(precision_score(labels, preds),average='micro'))
    print("Recall Score: {}".format(recall_score(labels, preds),average='micro'))
    print("Accuracy Score: {}".format(accuracy_score(labels, preds)))
    print("F1 Score: {}".format(f1_score(labels, preds)),average='micro')

### Dummy classifier

In [11]:
# dummy without SVD
dum_clf = DummyClassifier().fit(vector_df,y_train)
y_hat_train = dum_clf.predict(vector_df)
y_hat_test = dum_clf.predict(vector_df_test)

In [12]:
print(accuracy_score(y_hat_train,y_train))
print(accuracy_score(y_hat_test,y_test))

0.20072
0.20664


In [ ]:
## dummy with LSA/SVD and engineered feats.
dum_clf_2 = DummyClassifier()
dum_model_2 = dum_clf_2.fit(df_LSA,y_train)
y_hat_test_2 = dum_model_2.predict(df_LSA_test)

In [ ]:
print(accuracy_score(y_hat_test_2,y_test))

### Logistic Regression

In [13]:
logreg = LogisticRegression(fit_intercept = False, solver= 'newton-cg',multi_class='multinomial')
model_log = logreg.fit(vector_df,y_train)
y_hat_train_log = model_log.predict(vector_df)
y_hat_test_log = model_log.predict(vector_df_test)

In [14]:
print_metrics(y_hat_train_log,y_train)

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [ ]:
print_metrics(y_hat_test_log,y_test)

### Decision Tree

In [ ]:
## Decision Tree without SVD
dt_clf = DecisionTreeClassifier()
dt_clf_model = dt_clf.fit(vector_df,y_train)
y_hat_train_DT = dt_clf_model.predict(vector_df)
y_hat_test_DT = dt_clf_model.predict(vector_df_test)

In [ ]:
print(accuracy_score(y_hat_test_DT,y_test))

In [ ]:
## Decision Tree with SVD
dt_clf_2 = DecisionTreeClassifier()
dt_clf_model_2 = dt_clf_2.fit(df_LSA,y_train)
y_hat_test_DT_2 = dt_clf_model_2.predict(df_LSA_test)

In [ ]:
print(accuracy_score(y_hat_test_DT_2,y_test))

In [ ]:
## Decision Tree with SVD and engineered feats.
dt_clf_3 = DecisionTreeClassifier()
dt_clf_model_3 = dt_clf_3.fit(df_SVD_train,y_train)
y_hat_test_DT_3 = dt_clf_model_3.predict(df_SVD_test)

In [ ]:
print(accuracy_score(y_hat_test_DT_3,y_test))

### Naive Bayes

In [ ]:
# NB with tfidf
NB = MultinomialNB()
NB_model = NB.fit(vector_df, y_train)
y_hat_test_NB = NB.predict(vector_df_test)

In [ ]:
print(accuracy_score(y_hat_test_NB,y_test))

In [ ]:
# plot confusion matrix
cnf_matrix = confusion_matrix(y_test, y_hat_test_NB)

plt.figure()
plot_confusion_matrix(cnf_matrix, classes=[1,2,3,4,5],normalize=True,
                      title='Naive Bayes Confusion Matrix')

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    plot_confusion_matrix(cm, classes,normalize=False,
                          title='Confusion matrix',cmap=plt.cm.Blues)
    
    """
    #Add Normalization Option
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig('confusion')

In [ ]:
## NB with bag of words
NB = MultinomialNB()
NB_model = NB.fit(bow_vectors, y_train)
y_hat_test_NB_bow = NB.predict(bow_test_df)

In [ ]:
print(accuracy_score(y_hat_test_NB_bow,y_test))

In [ ]:
def plot_AUC_ROC(y_score,fpr,tpr):
    sns.set_style("darkgrid", {"axes.facecolor": ".9"})
    print('AUC: {}'.format(auc(fpr, tpr)))
    plt.figure(figsize=(10,8))
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve')
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.yticks([i/20.0 for i in range(21)])
    plt.xticks([i/20.0 for i in range(21)])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.show()